In [2]:
from detection.iterator import iterate_all, get_all_traces
from download.otf import process_otf
import obspy as op
from obspy.signal.trigger import plot_trigger, recursive_sta_lta, trigger_onset
import os
import numpy as np

tr_demo = op.read('outputs/raw_data_cache/20150115/XH_DR14/HHZ.mseed')[0]

# STA/LTA detector
def detect_stalta(tr = op.Trace):
    output_path = 'outputs/stalta_detections/'
    date = tr.stats.starttime.strftime('%Y%m%d')
    sta_name = tr.stats.station
    out_file = os.path.join(output_path, f'{date}/{sta_name}.csv')
    os.makedirs(os.path.dirname(out_file), exist_ok=True)

    tr = process_otf(tr, fs=50, pre_filt=[4, 5, 20, 22])

    # tr.trigger('classicstalta', sta=0.5, lta=10.0)
    out = recursive_sta_lta(tr.data, int(2 * tr.stats.sampling_rate), int(60.0 * tr.stats.sampling_rate))
    # plot_trigger(tr, out, 10, 8)
    onsets = trigger_onset(out,11, 8, 10)
    np.savetxt(out_file, onsets, delimiter=',')
    return onsets

def detect_stalta_fromtracepath(trace_path):
    tr = op.read(trace_path)[0]
    return detect_stalta(tr)

onsets = detect_stalta(tr_demo)

In [4]:
import multiprocessing as mp
from detection.stalta import detect_stalta_fromtracepath
from detection.iterator import get_all_traces, get_incomplete_traces

trace_list = get_incomplete_traces()
mp.freeze_support()
p = mp.Pool(processes = mp.cpu_count())
for result in p.imap_unordered(detect_stalta_fromtracepath, trace_list):
    # print(result)
    pass
p.close()
p.join()

Exception: ERROR 1 stalta: len(data) < nlta

In [18]:
# demo of network coincidence triggering
from obspy.signal.trigger import coincidence_trigger
import obspy as op

st = op.Stream()
files = ["outputs/raw_data_cache/20150115/XH_DR05/HHZ.mseed",
        "outputs/raw_data_cache/20150115/XH_DR06/HHZ.mseed",
        "outputs/raw_data_cache/20150115/XH_DR07/HHZ.mseed",
        "outputs/raw_data_cache/20150115/XH_DR08/HHZ.mseed",
        "outputs/raw_data_cache/20150115/XH_DR09/HHZ.mseed",
        "outputs/raw_data_cache/20150115/XH_DR10/HHZ.mseed",
        "outputs/raw_data_cache/20150115/XH_DR11/HHZ.mseed",
        "outputs/raw_data_cache/20150115/XH_DR12/HHZ.mseed",
        "outputs/raw_data_cache/20150115/XH_DR13/HHZ.mseed",
        "outputs/raw_data_cache/20150115/XH_DR14/HHZ.mseed",
        "outputs/raw_data_cache/20150115/XH_RS04/HHZ.mseed",
        "outputs/raw_data_cache/20150115/XH_RS05/HHZ.mseed"]

for f in files:
    tr = op.read(f)[0]
    tr_filt = process_otf(tr, fs=50, pre_filt=[4, 5, 20, 22])
    st += tr_filt

st2 = st.copy()
trig = coincidence_trigger('recstalta', 3.5, 1, st2, 3, sta=0.5, lta=10.0)